In [ ]:
import rasterio
import numpy as np
from math import sqrt

In [ ]:
bands = ["../Données/S1Fall/measurement/s1a-iw-grd-vv-20191011t223652-20191011t223717-029417-035883-001.tiff",
        "../Données/S1Spring/measurement/s1a-iw-grd-vh-20200408t223649-20200408t223714-032042-03b3ce-002.tiff",
        "../Données/S1Winter/measurement/s1a-iw-grd-vv-20200115t223649-20200115t223714-030817-038911-001.tiff",
        "../Données/S1Summer/measurement/s1a-iw-grd-vv-20200713t223654-20200713t223719-033442-03e00d-001.tiff"]

In [ ]:
def get_vectors(gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left):
    """Forme les vecteurs moyens entre 4 coordonnées GroundControlPoint"""
    norm_v1 = (gcp_top_left.col - gcp_top_right.col)
    v1 = np.array([(gcp_top_left.x - gcp_top_right.x + gcp_bottom_left.x - gcp_bottom_right.x) / (2 * norm_v1),
         (gcp_top_left.y - gcp_top_right.y + gcp_bottom_left.y - gcp_bottom_right.y) / (2 * norm_v1)])
    norm_v2 = (gcp_bottom_right.row - gcp_top_right.row)
    v2 = np.array([(gcp_bottom_right.x - gcp_top_right.x + gcp_bottom_left.x - gcp_top_left.x) / (2 * norm_v2),
         (gcp_bottom_right.y - gcp_top_right.y + gcp_bottom_left.y - gcp_top_left.y ) / (2 * norm_v2)])
    return v1, v2

In [ ]:
from osgeo import gdal
def create_gcps(list_gcps):
    """Fait une approximation des gcps en moyennant les gcps de toutes les bandes de saisons (prend comme point de départ le
    plus petit x et le plus petit y trouvé)"""
    firsts_x = []
    firsts_y = []
    for band in list_gcps:
        firsts_x.append(band[0].x)
        firsts_y.append(band[0].y)
    x_min = min(firsts_x)
    y_min = min(firsts_y)
    gcps = [rasterio.control.GroundControlPoint(row=0.0, col=0.0, x=x_min, y=y_min, id='1')]
    for i in range(1, len(list_gcps[0])):
        previous_x_average = 0
        previous_y_average = 0
        actual_x_average = 0
        actual_y_average = 0
        for band in list_gcps:
            previous_x_average += band[i-1].x
            previous_y_average += band[i-1].y
            actual_x_average += band[i].x
            actual_y_average += band[i].y
        x = gcps[i-1].x + (actual_x_average - previous_x_average) / len(list_gcps)
        y = gcps[i-1].y + (actual_y_average - previous_y_average) / len(list_gcps)  
        gcps.append(rasterio.control.GroundControlPoint(row=list_gcps[0][i].row, col=list_gcps[0][i].col,
                                                        x=x, y=y, id=str(i+1)))
    return gcps

In [ ]:
from sympy import symbols, Eq, solve
def get_offset(gcps, x_min, y_min):
    gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left = None, None, None, None
    for gcp in gcps:
        if gcp.row == 0.0 and gcp.col == 0.0:
            gcp_top_right = gcp
        elif gcp.row == 0.0 and gcp_top_left == None:
            gcp_top_left = gcp
        elif gcp.col <= 0.0 and gcp_bottom_right == None:
            gcp_bottom_right = gcp
        elif gcp_bottom_right != None:
            gcp_bottom_left = gcp
            break
    v1, v2 = get_vectors(gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left)
    origin = np.array([gcp_top_right.x, gcp_top_right.y])
    u1, u2 = symbols('u1 u2')
    eq1 = Eq(gcp_top_right.x - u1*v1[0] - u2*v2[0] - x_min, 0)
    eq2 = Eq(gcp_top_right.y - u1*v1[1] - u2*v2[1] - y_min, 0)
    sol = solve((eq1, eq2),(u1, u2))
    return round(sol[u1]), round(sol[u2])

In [ ]:
def translate_image(image, offset_x, offset_y):
    result = []
    for i in range(len(image)):
        if (i % 1000) == 0:
            print(i)
        if 0 <= (i + offset_y) < len(image):
            result_row = []
            for j in range(len(image[0])):
                if 0 <= (j + offset_x) < len(image[0]):
                    result_row.append(image[i + offset_y][j + offset_x])
                else:
                    result_row.append(0)
            result.append(result_row)
        else:
            result.append([0 for i in range(len(image[0]))])
    return np.array(result, dtype=np.uint16)

In [ ]:
def translate_bands(bands):
    list_gcps = []
    for band in bands:
        list_gcps.append(rasterio.open(band).gcps[0])
    gcps = create_gcps(list_gcps)
    for band in bands:
        image = rasterio.open(band)
        offset_x, offset_y = get_offset(image.gcps[0], gcps[0].x, gcps[0].y)
        translated = translate_image(image.read(1), offset_x, offset_y)
        new_image = rasterio.open(band[:-5] + '_translated.tiff', 'w', driver='Gtiff', 
                          width=len(image.read(1)[0]), height=len(image.read(1)), count=1, crs=image.gcps[1], gcps=gcps,
                          transform=image.transform,
                          dtype=np.uint16
                         )
        image.close()
        new_image.write(translated, 1)
        #del translated
        new_image.close()
        print("Fin", band)

In [ ]:
translate_bands(bands)